In [49]:
import pandas as pd
from pymongo import MongoClient

In [50]:
keyword_combinations = pd.read_excel('keywords.xlsx')
print(keyword_combinations)

              Category                      Combination
0               CARBON     Carbon AND Carbon neutrality
1               CARBON             Carbon AND Emissions
2               CARBON             Carbon AND Abatement
3               CARBON  Carbon neutrality AND Emissions
4               CARBON  Carbon neutrality AND Abatement
..                 ...                              ...
78  NATIONAL ECONOMICS                  Trade AND dairy
79              HEALTH     Nutrition health AND Obesity
80              HEALTH      Nutrition health OR Obesity
81              HEALTH       Nutrition health AND dairy
82              HEALTH                Obesity AND dairy

[83 rows x 2 columns]


# Original Logic

In [51]:
def find_top_documents_with_keyword_frequencies_or(db_name, tdm_collection_name, keywords, top_n=5, mongo_uri="mongodb://localhost:27017/"):
    client = MongoClient(mongo_uri)
    db = client[db_name]
    tdm_collection = db[tdm_collection_name]
    keywords = [keyword.replace(' ', '_') for keyword in keywords]
    
    pipeline = [
        {
            '$project': {
                'File Name': 1,
                **{keyword: {'$ifNull': [f'$Term Document Matrix.{keyword}', 0]} for keyword in keywords}
            }
        },
        {
            '$addFields': {
                'TotalFrequency': {
                    '$sum': [f'${keyword}' for keyword in keywords]
                }
            }
        },
        {
            '$match': {
                'TotalFrequency': {'$gt': 0}
            }
        },
        {'$sort': {'TotalFrequency': -1}},
        {'$limit': top_n},
        {
            '$project': {
                'File Name': 1,
                **{keyword: 1 for keyword in keywords}
            }
        }
    ]
    
    print("Pipeline:", pipeline)
    results = list(tdm_collection.aggregate(pipeline))
    print("Results:", results)
    
    for result in results:
        print(f"File Name: {result['File Name']}")
        for keyword in keywords:
            print(f"  {keyword}: {result[keyword]}")
    
    return results

def find_top_documents_with_keyword_frequencies_and(db_name, tdm_collection_name, keywords, top_n=5, mongo_uri="mongodb://localhost:27017/"):
    client = MongoClient(mongo_uri)
    db = client[db_name]
    tdm_collection = db[tdm_collection_name]
    keywords = [keyword.replace(' ', '_') for keyword in keywords]
    
    pipeline = [
        {
            '$project': {
                'File Name': 1,
                **{keyword: {'$ifNull': [f'$Term Document Matrix.{keyword}', 0]} for keyword in keywords}
            }
        },
        {
            '$match': {
                **{keyword: {'$gt': 0} for keyword in keywords}
            }
        },
        {
            '$addFields': {
                'TotalFrequency': {
                    '$sum': [f'${keyword}' for keyword in keywords]
                }
            }
        },
        {'$sort': {'TotalFrequency': -1}},
        {'$limit': top_n},
        {
            '$project': {
                'File Name': 1,
                **{keyword: 1 for keyword in keywords}
            }
        }
    ]
    
    print("Pipeline:", pipeline)
    results = list(tdm_collection.aggregate(pipeline))
    print("Results:", results)
    for result in results:
        print(f"File Name: {result['File Name']}")
        for keyword in keywords:
            print(f"  {keyword}: {result[keyword]}")
    
    return results

def function_call(keywords):
    keyword_list = []
    
    if 'OR' in keywords:
        print('OR')
        keyword_list = [keyword.strip().lower() for keyword in keywords.split('OR')]
        top_documents = find_top_documents_with_keyword_frequencies_or('transcripts', 'complete_documents', keyword_list, top_n=5)
    elif 'AND' in keywords:
        print('AND')
        keyword_list = [keyword.strip().lower() for keyword in keywords.split('AND')]
        print(keyword_list)
        top_documents = find_top_documents_with_keyword_frequencies_and('transcripts', 'complete_documents', keyword_list, top_n=5)
    else:
        keyword_list.insert(0,str(keywords))
        top_documents = find_top_documents_with_keyword_frequencies_or('transcripts', 'complete_documents', keyword_list, top_n=5)
    
    return top_documents

In [52]:
keyword_combinations['relevant_doc'] = keyword_combinations['Combination'].apply(function_call)

AND
['carbon', 'carbon neutrality']
Pipeline: [{'$project': {'File Name': 1, 'carbon': {'$ifNull': ['$Term Document Matrix.carbon', 0]}, 'carbon_neutrality': {'$ifNull': ['$Term Document Matrix.carbon_neutrality', 0]}}}, {'$match': {'carbon': {'$gt': 0}, 'carbon_neutrality': {'$gt': 0}}}, {'$addFields': {'TotalFrequency': {'$sum': ['$carbon', '$carbon_neutrality']}}}, {'$sort': {'TotalFrequency': -1}}, {'$limit': 5}, {'$project': {'File Name': 1, 'carbon': 1, 'carbon_neutrality': 1}}]
Results: []
AND
['carbon', 'emissions']
Pipeline: [{'$project': {'File Name': 1, 'carbon': {'$ifNull': ['$Term Document Matrix.carbon', 0]}, 'emissions': {'$ifNull': ['$Term Document Matrix.emissions', 0]}}}, {'$match': {'carbon': {'$gt': 0}, 'emissions': {'$gt': 0}}}, {'$addFields': {'TotalFrequency': {'$sum': ['$carbon', '$emissions']}}}, {'$sort': {'TotalFrequency': -1}}, {'$limit': 5}, {'$project': {'File Name': 1, 'carbon': 1, 'emissions': 1}}]
Results: [{'_id': ObjectId('670521f275d9daeff229ce01'), 

In [53]:
print(keyword_combinations)

              Category                      Combination  \
0               CARBON     Carbon AND Carbon neutrality   
1               CARBON             Carbon AND Emissions   
2               CARBON             Carbon AND Abatement   
3               CARBON  Carbon neutrality AND Emissions   
4               CARBON  Carbon neutrality AND Abatement   
..                 ...                              ...   
78  NATIONAL ECONOMICS                  Trade AND dairy   
79              HEALTH     Nutrition health AND Obesity   
80              HEALTH      Nutrition health OR Obesity   
81              HEALTH       Nutrition health AND dairy   
82              HEALTH                Obesity AND dairy   

                                         relevant_doc  
0                                                  []  
1   [{'_id': 670521f275d9daeff229ce01, 'File Name'...  
2   [{'_id': 670521f275d9daeff229ce43, 'File Name'...  
3                                                  []  
4          

In [54]:
keyword_combinations.to_excel('keyword_combinations_result.xlsx', index = False)

# Incomplete Words

In [55]:
from pymongo import MongoClient
import re

def find_top_documents_with_keyword_frequencies_or_inc(db_name, tdm_collection_name, keywords, top_n=5, mongo_uri="mongodb://localhost:27017/"):
    client = MongoClient(mongo_uri)
    db = client[db_name]
    tdm_collection = db[tdm_collection_name]
    
    # Prepare keywords: replace spaces with underscores and handle wildcards
    processed_keywords = []
    wildcard_patterns = []
    for keyword in keywords:
        keyword = keyword.replace(' ', '_')
        if '*' in keyword:
            # Convert wildcard (*) to regex pattern
            pattern = re.compile(f"^(?!.*[_\\d]).*{keyword.replace('*', '.*')}$")
            wildcard_patterns.append(pattern)
        else:
            processed_keywords.append(keyword)
    
    # Fetch unique terms using aggregation (avoids 16MB limit)
    pipeline = [
        {'$project': {'terms': {'$objectToArray': '$Term Document Matrix'}}},
        {'$unwind': '$terms'},
        {'$group': {'_id': None, 'unique_terms': {'$addToSet': '$terms.k'}}},
        {'$project': {'_id': 0, 'unique_terms': 1}}
    ]
    result = list(tdm_collection.aggregate(pipeline))
    if result:
        all_terms = result[0]['unique_terms']
    else:
        all_terms = []
    
    # Match wildcard patterns with terms in the collection
    for pattern in wildcard_patterns:
        matched_terms = [term for term in all_terms if pattern.match(term)]
        processed_keywords.extend(matched_terms)
    
    # Deduplicate the final list of keywords
    processed_keywords = list(set(processed_keywords))
    
    # Build the pipeline
    pipeline = [
        {
            '$project': {
                'File Name': 1,
                **{keyword: {'$ifNull': [f'$Term Document Matrix.{keyword}', 0]} for keyword in processed_keywords}
            }
        },
        {
            '$addFields': {
                'TotalFrequency': {
                    '$sum': [f'${keyword}' for keyword in processed_keywords]
                }
            }
        },
        {
            '$match': {
                'TotalFrequency': {'$gt': 0}
            }
        },
        {'$sort': {'TotalFrequency': -1}},
        {'$limit': top_n},
        {
            '$project': {
                'File Name': 1,
                **{keyword: 1 for keyword in processed_keywords}
            }
        }
    ]
    
    print("Pipeline:", pipeline)
    results = list(tdm_collection.aggregate(pipeline))
    print("Results:", results)
    
    for result in results:
        print(f"File Name: {result['File Name']}")
        for keyword in processed_keywords:
            print(f"  {keyword}: {result.get(keyword, 0)}")
    
    return results


In [56]:
from pymongo import MongoClient
import re

def find_top_documents_with_keyword_frequencies_and_inc(db_name, tdm_collection_name, keywords, top_n=5, mongo_uri="mongodb://localhost:27017/"):
    client = MongoClient(mongo_uri)
    db = client[db_name]
    tdm_collection = db[tdm_collection_name]
    
    # Prepare keywords: replace spaces with underscores and handle wildcards
    processed_keywords = []
    wildcard_patterns = []
    for keyword in keywords:
        keyword = keyword.replace(' ', '_')
        if '*' in keyword:
            # Convert wildcard (*) to regex pattern
            pattern = re.compile(f"^(?!.*[_\\d]).*{keyword.replace('*', '.*')}$")
            wildcard_patterns.append(pattern)
        else:
            processed_keywords.append(keyword)
    
    # Fetch unique terms using aggregation (avoids 16MB limit)
    pipeline = [
        {'$project': {'terms': {'$objectToArray': '$Term Document Matrix'}}},
        {'$unwind': '$terms'},
        {'$group': {'_id': None, 'unique_terms': {'$addToSet': '$terms.k'}}},
        {'$project': {'_id': 0, 'unique_terms': 1}}
    ]
    result = list(tdm_collection.aggregate(pipeline))
    if result:
        all_terms = result[0]['unique_terms']
    else:
        all_terms = []
    
    # Match wildcard patterns with terms in the collection
    for pattern in wildcard_patterns:
        matched_terms = [term for term in all_terms if pattern.match(term)]
        processed_keywords.extend(matched_terms)
    
    # Deduplicate the final list of keywords
    processed_keywords = list(set(processed_keywords))
    
    # Build the pipeline
    pipeline = [
        {
            '$project': {
                'File Name': 1,
                **{keyword: {'$ifNull': [f'$Term Document Matrix.{keyword}', 0]} for keyword in processed_keywords}
            }
        },
        {
            '$match': {
                **{keyword: {'$gt': 0} for keyword in processed_keywords}
            }
        },
        {
            '$addFields': {
                'TotalFrequency': {
                    '$sum': [f'${keyword}' for keyword in processed_keywords]
                }
            }
        },
        {'$sort': {'TotalFrequency': -1}},
        {'$limit': top_n},
        {
            '$project': {
                'File Name': 1,
                **{keyword: 1 for keyword in processed_keywords}
            }
        }
    ]
    
    print("Pipeline:", pipeline)
    results = list(tdm_collection.aggregate(pipeline))
    print("Results:", results)
    for result in results:
        print(f"File Name: {result['File Name']}")
        for keyword in processed_keywords:
            print(f"  {keyword}: {result.get(keyword, 0)}")
    
    return results


In [57]:
def function_call_inc(keywords):
    keyword_list = []
    
    if 'OR' in keywords:
        print('OR')
        keyword_list = [keyword.strip().lower() for keyword in keywords.split('OR')]
        top_documents = find_top_documents_with_keyword_frequencies_or_inc('transcripts', 'complete_documents', keyword_list, top_n=1000)
    elif 'AND' in keywords:
        print('AND')
        keyword_list = [keyword.strip().lower() for keyword in keywords.split('AND')]
        print(keyword_list)
        top_documents = find_top_documents_with_keyword_frequencies_and_inc('transcripts', 'complete_documents', keyword_list, top_n=1000)
    else:
        keyword_list.insert(0,str(keywords))
        top_documents = find_top_documents_with_keyword_frequencies_or_inc('transcripts', 'complete_documents', keyword_list, top_n=1000)
    
    return top_documents

# Dairy and Carbon

In [58]:
kw = 'dairy* OR milk OR cattle'
dairy_doc = function_call_inc(kw)
print(dairy_doc)

OR
Pipeline: [{'$project': {'File Name': 1, 'cattle': {'$ifNull': ['$Term Document Matrix.cattle', 0]}, 'dairygold': {'$ifNull': ['$Term Document Matrix.dairygold', 0]}, 'dairying': {'$ifNull': ['$Term Document Matrix.dairying', 0]}, 'dairy-beef': {'$ifNull': ['$Term Document Matrix.dairy-beef', 0]}, 'dairy-cross': {'$ifNull': ['$Term Document Matrix.dairy-cross', 0]}, 'dairy-based': {'$ifNull': ['$Term Document Matrix.dairy-based', 0]}, 'dairy-bred': {'$ifNull': ['$Term Document Matrix.dairy-bred', 0]}, 'dairy': {'$ifNull': ['$Term Document Matrix.dairy', 0]}, 'milk': {'$ifNull': ['$Term Document Matrix.milk', 0]}, 'dairymaster': {'$ifNull': ['$Term Document Matrix.dairymaster', 0]}, 'non-dairy': {'$ifNull': ['$Term Document Matrix.non-dairy', 0]}}}, {'$addFields': {'TotalFrequency': {'$sum': ['$cattle', '$dairygold', '$dairying', '$dairy-beef', '$dairy-cross', '$dairy-based', '$dairy-bred', '$dairy', '$milk', '$dairymaster', '$non-dairy']}}}, {'$match': {'TotalFrequency': {'$gt': 0}}

In [59]:
kw_carbon = 'carbon OR neutrality OR emissions OR abatement'
carbon_doc = function_call_inc(kw_carbon)
print(carbon_doc)

OR
Pipeline: [{'$project': {'File Name': 1, 'abatement': {'$ifNull': ['$Term Document Matrix.abatement', 0]}, 'neutrality': {'$ifNull': ['$Term Document Matrix.neutrality', 0]}, 'emissions': {'$ifNull': ['$Term Document Matrix.emissions', 0]}, 'carbon': {'$ifNull': ['$Term Document Matrix.carbon', 0]}}}, {'$addFields': {'TotalFrequency': {'$sum': ['$abatement', '$neutrality', '$emissions', '$carbon']}}}, {'$match': {'TotalFrequency': {'$gt': 0}}}, {'$sort': {'TotalFrequency': -1}}, {'$limit': 1000}, {'$project': {'File Name': 1, 'abatement': 1, 'neutrality': 1, 'emissions': 1, 'carbon': 1}}]
Results: [{'_id': ObjectId('670521f275d9daeff229ce01'), 'File Name': 'Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 17 Nov 2021.pdf', 'abatement': 0, 'neutrality': 4, 'emissions': 77, 'carbon': 274}, {'_id': ObjectId('670521f275d9daeff229ce43'), 'File Name': 'Joint Committee on Climate Action debate - Wednesday, 12 Dec 2018.pdf', 'abatement': 12, 'neutrality': 0, 'emission

In [60]:
def find_matching_files(doc_list_other, doc_list_dairy):
    file_names_set_dairy = {doc['File Name'] for doc in doc_list_dairy}
    matching_records = [other_doc for other_doc in doc_list_other if other_doc['File Name'] in file_names_set_dairy]
    return matching_records

In [61]:
matching_records = find_matching_files(carbon_doc, dairy_doc)

print("Matching Records:", len(matching_records))
print(len(carbon_doc))
print(len(dairy_doc))

Matching Records: 377
511
491


In [62]:
dairy_carbon_doc = [doc['File Name'] for doc in matching_records]
print(dairy_carbon_doc)

['Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 17 Nov 2021.pdf', 'Joint Committee on Climate Action debate - Wednesday, 12 Dec 2018.pdf', 'Dáil Éireann debate - Wednesday, 10 Jun 2020.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 20 Jul 2022.pdf', 'Dáil Éireann debate - Wednesday, 6 Apr 2022.pdf', 'Joint Committee on Climate Action debate - Wednesday, 12 Sep 2018.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Tuesday, 6 Mar 2018.pdf', 'Joint Committee on Environment and Climate Action debate - Wednesday, 12 Jan 2022.pdf', 'Dáil Éireann debate - Wednesday, 21 Apr 2021.pdf', 'Dáil Éireann debate - Thursday, 7 Dec 2017.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Thursday, 24 Mar 2022.pdf', 'Joint Committee on Climate Action debate - Wednesday, 7 Nov 2018.pdf', 'Joint Committee on Environment and Climate Action debate - Tuesday, 15 Jun 2021.pdf', 'Dáil Éireann debate - Tuesday, 8 Oct 20

In [63]:
dairy_carbon_doc_with_frequencies = pd.DataFrame(matching_records)
dairy_carbon_doc_with_frequencies['Query Category'] = "Dairy and Carbon"
dairy_carbon_doc_with_frequencies = dairy_carbon_doc_with_frequencies.loc[:, (dairy_carbon_doc_with_frequencies != 0).any()]
print(dairy_carbon_doc_with_frequencies)

                          _id  \
0    670521f275d9daeff229ce01   
1    670521f275d9daeff229ce43   
2    670521f275d9daeff229cf31   
3    670521f275d9daeff229ce07   
4    670521f275d9daeff229cf94   
..                        ...   
372  670521f275d9daeff229ce9d   
373  670521f275d9daeff229cf8b   
374  670521f275d9daeff229ce9e   
375  670521f275d9daeff229cdfb   
376  670521f275d9daeff229cfd9   

                                             File Name  abatement  neutrality  \
0    Joint Committee on Agriculture, Food and the M...          0           4   
1    Joint Committee on Climate Action debate - Wed...         12           0   
2     Dáil Éireann debate - Wednesday, 10 Jun 2020.pdf          4           4   
3    Joint Committee on Agriculture, Food and the M...          3           4   
4      Dáil Éireann debate - Wednesday, 6 Apr 2022.pdf          0          12   
..                                                 ...        ...         ...   
372    Dáil Éireann debate - Thursda

In [64]:
dairy_carbon_doc_with_frequencies["Total Freq"] = dairy_carbon_doc_with_frequencies.select_dtypes(include=['number']).sum(axis=1)
dairy_carbon_doc_with_frequencies['Year'] = dairy_carbon_doc_with_frequencies['File Name'].str.extract(r'(\d{4})')
print(dairy_carbon_doc_with_frequencies)

                          _id  \
0    670521f275d9daeff229ce01   
1    670521f275d9daeff229ce43   
2    670521f275d9daeff229cf31   
3    670521f275d9daeff229ce07   
4    670521f275d9daeff229cf94   
..                        ...   
372  670521f275d9daeff229ce9d   
373  670521f275d9daeff229cf8b   
374  670521f275d9daeff229ce9e   
375  670521f275d9daeff229cdfb   
376  670521f275d9daeff229cfd9   

                                             File Name  abatement  neutrality  \
0    Joint Committee on Agriculture, Food and the M...          0           4   
1    Joint Committee on Climate Action debate - Wed...         12           0   
2     Dáil Éireann debate - Wednesday, 10 Jun 2020.pdf          4           4   
3    Joint Committee on Agriculture, Food and the M...          3           4   
4      Dáil Éireann debate - Wednesday, 6 Apr 2022.pdf          0          12   
..                                                 ...        ...         ...   
372    Dáil Éireann debate - Thursda

## Term Coverage

In [65]:
numeric_columns = dairy_carbon_doc_with_frequencies.select_dtypes(include=['number'])
dairy_carbon_doc_with_frequencies["Term Coverage"] = (numeric_columns > 0).sum(axis=1)

In [66]:
carbon_sorted_by_coverage = dairy_carbon_doc_with_frequencies.sort_values(by="Term Coverage", ascending=False)
print(carbon_sorted_by_coverage[["File Name", "Term Coverage"]].head())

                                            File Name  Term Coverage
18  Joint Committee on Environment and Climate Act...              5
6   Joint Committee on Agriculture, Food and the M...              5
17  Joint Committee on Climate Action debate - Wed...              5
93   Dáil Éireann debate - Wednesday, 27 Mar 2019.pdf              5
84  Seanad Éireann debate - Thursday, 14 Feb 2019.pdf              5


In [67]:
threshold = 3
carbon_relevant_documents = carbon_sorted_by_coverage[carbon_sorted_by_coverage["Term Coverage"] >= threshold]
carbon_relevant_documents.to_excel("carbon_relevant_documents.xlsx", index=False)

# Dairy and Water

In [68]:
kw_water = 'water OR waterways OR rivers OR run-off OR nitr* OR derogation OR eutrophication OR alga* OR bloom OR effluent OR discharge'
water_doc = function_call_inc(kw_water)
print(water_doc)

OR
Pipeline: [{'$project': {'File Name': 1, 'nitrogen-efficient': {'$ifNull': ['$Term Document Matrix.nitrogen-efficient', 0]}, 'algal': {'$ifNull': ['$Term Document Matrix.algal', 0]}, 'algae': {'$ifNull': ['$Term Document Matrix.algae', 0]}, 'nitro': {'$ifNull': ['$Term Document Matrix.nitro', 0]}, 'bloom': {'$ifNull': ['$Term Document Matrix.bloom', 0]}, 'nitrogen': {'$ifNull': ['$Term Document Matrix.nitrogen', 0]}, 'nitrates': {'$ifNull': ['$Term Document Matrix.nitrates', 0]}, 'nitrous': {'$ifNull': ['$Term Document Matrix.nitrous', 0]}, 'amalgama': {'$ifNull': ['$Term Document Matrix.amalgama', 0]}, 'discharge': {'$ifNull': ['$Term Document Matrix.discharge', 0]}, 'nitrogen-fixing': {'$ifNull': ['$Term Document Matrix.nitrogen-fixing', 0]}, 'amalgamated': {'$ifNull': ['$Term Document Matrix.amalgamated', 0]}, 'run-off': {'$ifNull': ['$Term Document Matrix.run-off', 0]}, 'dualgais': {'$ifNull': ['$Term Document Matrix.dualgais', 0]}, 'rivers': {'$ifNull': ['$Term Document Matrix.

In [69]:
matching_records_water = find_matching_files(water_doc, dairy_doc)

print("Matching Records:", len(matching_records_water))
print(len(water_doc))
print(len(dairy_doc))

Matching Records: 408
557
491


In [70]:
dairy_water_doc = [doc['File Name'] for doc in matching_records_water]
print(dairy_water_doc)

['Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 19 Jul 2023.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 13 Oct 2021.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 14 Sep 2022.pdf', 'Dáil Éireann debate - Wednesday, 19 May 2021.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 5 Jul 2023.pdf', 'Joint Committee on Housing, Local Government and Heritage debate - Thursday, 7 Apr 2022.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 21 Feb 2024.pdf', 'Dáil Éireann debate - Wednesday, 9 Nov 2016.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Friday, 15 Sep 2023.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 20 Mar 2024.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 6 Mar 2024.pdf', 'Joint Committee on Agriculture, Food and the Marine debate - Wednesday, 14 Fe

In [71]:
dairy_water_doc_with_frequencies = pd.DataFrame(matching_records_water)
dairy_water_doc_with_frequencies['Query Category'] = "Dairy and Water"
dairy_water_doc_with_frequencies = dairy_water_doc_with_frequencies.loc[:, (dairy_water_doc_with_frequencies != 0).any()]
print(dairy_water_doc_with_frequencies)

                          _id  \
0    670521f275d9daeff229ce04   
1    670521f275d9daeff229cdfa   
2    670521f275d9daeff229cdfd   
3    670521f275d9daeff229cf58   
4    670521f275d9daeff229ce1e   
..                        ...   
403  670521f275d9daeff229cddc   
404  670521f275d9daeff229cdd7   
405  670521f275d9daeff229cde9   
406  670521f275d9daeff229ceb9   
407  670521f275d9daeff229d01d   

                                             File Name  nitrogen-efficient  \
0    Joint Committee on Agriculture, Food and the M...                   0   
1    Joint Committee on Agriculture, Food and the M...                   0   
2    Joint Committee on Agriculture, Food and the M...                   0   
3     Dáil Éireann debate - Wednesday, 19 May 2021.pdf                   0   
4    Joint Committee on Agriculture, Food and the M...                   0   
..                                                 ...                 ...   
403  Joint Committee on Agriculture, Food and the M...   

In [72]:
dairy_water_doc_with_frequencies["Total Freq"] = dairy_water_doc_with_frequencies.select_dtypes(include=['number']).sum(axis=1)
dairy_water_doc_with_frequencies['Year'] = dairy_water_doc_with_frequencies['File Name'].str.extract(r'(\d{4})')
print(dairy_water_doc_with_frequencies)

                          _id  \
0    670521f275d9daeff229ce04   
1    670521f275d9daeff229cdfa   
2    670521f275d9daeff229cdfd   
3    670521f275d9daeff229cf58   
4    670521f275d9daeff229ce1e   
..                        ...   
403  670521f275d9daeff229cddc   
404  670521f275d9daeff229cdd7   
405  670521f275d9daeff229cde9   
406  670521f275d9daeff229ceb9   
407  670521f275d9daeff229d01d   

                                             File Name  nitrogen-efficient  \
0    Joint Committee on Agriculture, Food and the M...                   0   
1    Joint Committee on Agriculture, Food and the M...                   0   
2    Joint Committee on Agriculture, Food and the M...                   0   
3     Dáil Éireann debate - Wednesday, 19 May 2021.pdf                   0   
4    Joint Committee on Agriculture, Food and the M...                   0   
..                                                 ...                 ...   
403  Joint Committee on Agriculture, Food and the M...   

## Term Coverage

In [73]:
numeric_columns_water = dairy_water_doc_with_frequencies.select_dtypes(include=['number'])
dairy_water_doc_with_frequencies["Term Coverage"] = (numeric_columns > 0).sum(axis=1)

In [74]:
water_sorted_by_coverage = dairy_water_doc_with_frequencies.sort_values(by="Term Coverage", ascending=False)
print(water_sorted_by_coverage[["File Name", "Term Coverage"]].head())

                                            File Name  Term Coverage
18     Dáil Éireann debate - Tuesday, 20 Nov 2018.pdf            5.0
6   Joint Committee on Agriculture, Food and the M...            5.0
17    Dáil Éireann debate - Thursday, 12 Jul 2018.pdf            5.0
93  Joint Committee on Environment and Climate Act...            5.0
84   Dáil Éireann debate - Wednesday, 31 May 2017.pdf            5.0


In [75]:
threshold = 3
water_relevant_documents = water_sorted_by_coverage[water_sorted_by_coverage["Term Coverage"] >= threshold]
water_relevant_documents.to_excel("water_relevant_documents.xlsx", index=False)

# Dairy and Livelihood

In [76]:
kw_livelihood = 'herit* OR succession OR rural development OR community OR social sustainability OR society OR economic viability'
livelihood_doc = function_call_inc(kw_livelihood)
print(livelihood_doc)

OR
Pipeline: [{'$project': {'File Name': 1, 'inheriting': {'$ifNull': ['$Term Document Matrix.inheriting', 0]}, 'rural_development': {'$ifNull': ['$Term Document Matrix.rural_development', 0]}, 'social_sustainability': {'$ifNull': ['$Term Document Matrix.social_sustainability', 0]}, 'anti-heritage': {'$ifNull': ['$Term Document Matrix.anti-heritage', 0]}, 'inherited': {'$ifNull': ['$Term Document Matrix.inherited', 0]}, 'heritability': {'$ifNull': ['$Term Document Matrix.heritability', 0]}, 'inherit': {'$ifNull': ['$Term Document Matrix.inherit', 0]}, 'economic_viability': {'$ifNull': ['$Term Document Matrix.economic_viability', 0]}, 'inheritances': {'$ifNull': ['$Term Document Matrix.inheritances', 0]}, 'thatcherite': {'$ifNull': ['$Term Document Matrix.thatcherite', 0]}, 'inherits': {'$ifNull': ['$Term Document Matrix.inherits', 0]}, 'community': {'$ifNull': ['$Term Document Matrix.community', 0]}, 'heritage': {'$ifNull': ['$Term Document Matrix.heritage', 0]}, 'succession': {'$ifNul

In [77]:
matching_records_livelihood = find_matching_files(livelihood_doc, dairy_doc)

print("Matching Records:", len(matching_records_livelihood))
print(len(livelihood_doc))
print(len(dairy_doc))

Matching Records: 458
649
491


In [78]:
dairy_livelihood_doc = [doc['File Name'] for doc in matching_records_livelihood]
print(dairy_livelihood_doc)

['Dáil Éireann debate - Thursday, 23 Feb 2023.pdf', 'Dáil Éireann debate - Wednesday, 28 Apr 2021.pdf', 'Dáil Éireann debate - Wednesday, 4 Nov 2015.pdf', 'Dáil Éireann debate - Thursday, 8 Jul 2021.pdf', 'Dáil Éireann debate - Wednesday, 12 Jul 2017.pdf', 'Dáil Éireann debate - Thursday, 25 Nov 2021.pdf', 'Dáil Éireann debate - Wednesday, 12 Oct 2022.pdf', 'Dáil Éireann debate - Wednesday, 16 Jun 2021.pdf', 'Dáil Éireann debate - Thursday, 15 Jul 2021.pdf', 'Dáil Éireann debate - Thursday, 16 Feb 2023.pdf', 'Dáil Éireann debate - Wednesday, 14 Jun 2023.pdf', 'Dáil Éireann debate - Wednesday, 27 Sep 2023.pdf', 'Dáil Éireann debate - Wednesday, 31 May 2017.pdf', 'Dáil Éireann debate - Wednesday, 1 Mar 2023.pdf', 'Dáil Éireann debate - Thursday, 21 May 2020.pdf', 'Dáil Éireann debate - Wednesday, 1 Mar 2017.pdf', 'Dáil Éireann debate - Thursday, 15 Dec 2022.pdf', 'Dáil Éireann debate - Wednesday, 4 Oct 2023.pdf', 'Dáil Éireann debate - Thursday, 14 Dec 2023.pdf', 'Dáil Éireann debate - T

In [79]:
dairy_livelihood_doc_with_frequencies = pd.DataFrame(matching_records_livelihood)
dairy_livelihood_doc_with_frequencies['Query Category'] = "Dairy and Livelihood"
dairy_livelihood_doc_with_frequencies = dairy_livelihood_doc_with_frequencies.loc[:, (dairy_livelihood_doc_with_frequencies != 0).any()]
print(dairy_livelihood_doc_with_frequencies)

                          _id  \
0    670521f275d9daeff229cea6   
1    670521f275d9daeff229cf76   
2    670521f275d9daeff229cf8f   
3    670521f275d9daeff229ced4   
4    670521f275d9daeff229cf3c   
..                        ...   
453  670521f275d9daeff229cdfe   
454  670521f275d9daeff229cdd8   
455  670521f275d9daeff229ce06   
456  670521f275d9daeff229cdbe   
457  670521f275d9daeff229cdb7   

                                             File Name  inheriting  \
0      Dáil Éireann debate - Thursday, 23 Feb 2023.pdf           0   
1     Dáil Éireann debate - Wednesday, 28 Apr 2021.pdf           0   
2      Dáil Éireann debate - Wednesday, 4 Nov 2015.pdf           0   
3       Dáil Éireann debate - Thursday, 8 Jul 2021.pdf           0   
4     Dáil Éireann debate - Wednesday, 12 Jul 2017.pdf           0   
..                                                 ...         ...   
453  Joint Committee on Agriculture, Food and the M...           0   
454  Joint Committee on Agriculture, Food a

In [80]:
dairy_livelihood_doc_with_frequencies["Total Freq"] = dairy_livelihood_doc_with_frequencies.select_dtypes(include=['number']).sum(axis=1)
dairy_livelihood_doc_with_frequencies['Year'] = dairy_livelihood_doc_with_frequencies['File Name'].str.extract(r'(\d{4})')
print(dairy_livelihood_doc_with_frequencies)

                          _id  \
0    670521f275d9daeff229cea6   
1    670521f275d9daeff229cf76   
2    670521f275d9daeff229cf8f   
3    670521f275d9daeff229ced4   
4    670521f275d9daeff229cf3c   
..                        ...   
453  670521f275d9daeff229cdfe   
454  670521f275d9daeff229cdd8   
455  670521f275d9daeff229ce06   
456  670521f275d9daeff229cdbe   
457  670521f275d9daeff229cdb7   

                                             File Name  inheriting  \
0      Dáil Éireann debate - Thursday, 23 Feb 2023.pdf           0   
1     Dáil Éireann debate - Wednesday, 28 Apr 2021.pdf           0   
2      Dáil Éireann debate - Wednesday, 4 Nov 2015.pdf           0   
3       Dáil Éireann debate - Thursday, 8 Jul 2021.pdf           0   
4     Dáil Éireann debate - Wednesday, 12 Jul 2017.pdf           0   
..                                                 ...         ...   
453  Joint Committee on Agriculture, Food and the M...           0   
454  Joint Committee on Agriculture, Food a

## Term Coverage

In [81]:
livelihood_columns_water = dairy_livelihood_doc_with_frequencies.select_dtypes(include=['number'])
dairy_livelihood_doc_with_frequencies["Term Coverage"] = (numeric_columns > 0).sum(axis=1)

In [82]:
livelihood_sorted_by_coverage = dairy_livelihood_doc_with_frequencies.sort_values(by="Term Coverage", ascending=False)
print(dairy_livelihood_doc_with_frequencies[["File Name", "Term Coverage"]].head())

                                          File Name  Term Coverage
0   Dáil Éireann debate - Thursday, 23 Feb 2023.pdf            4.0
1  Dáil Éireann debate - Wednesday, 28 Apr 2021.pdf            4.0
2   Dáil Éireann debate - Wednesday, 4 Nov 2015.pdf            5.0
3    Dáil Éireann debate - Thursday, 8 Jul 2021.pdf            5.0
4  Dáil Éireann debate - Wednesday, 12 Jul 2017.pdf            4.0


In [83]:
threshold = 3
livelihood_relevant_documents = livelihood_sorted_by_coverage[livelihood_sorted_by_coverage["Term Coverage"] >= threshold]
livelihood_relevant_documents.to_excel("livelihood_relevant_documents.xlsx", index=False)

## Combined Term Coverage

In [84]:
print({len(carbon_relevant_documents['File Name'])}, {len(water_relevant_documents['File Name'])}, {len(livelihood_relevant_documents['File Name'])})
combined_term_docs = list(
    set(
        carbon_relevant_documents['File Name'].tolist() +
        water_relevant_documents['File Name'].tolist() +
        livelihood_relevant_documents['File Name'].tolist()
    )
)

print(len(combined_term_docs))
#print(combined_term_docs)  

{282} {282} {282}
409


In [85]:
combined_term_docs = pd.DataFrame(combined_term_docs)
combined_term_docs.to_excel(r'TermCoverageListCombined.xlsx')

In [86]:
common_carbon_relevant_documents = [
    doc for _, doc in carbon_relevant_documents.iterrows()
    if doc['File Name'] in combined_term_docs
]
print(len(common_carbon_relevant_documents)) 

ccarbon = carbon_relevant_documents['File Name'].tolist()
print(len(ccarbon))
ccarbon_1 = [c for c in ccarbon if c in combined_term_docs]
print(len(ccarbon_1))

0
282
0


In [87]:
common_water_relevant_documents = [
    doc for _, doc in water_relevant_documents.iterrows()
    if doc['File Name'] in combined_term_docs
]
print(len(common_water_relevant_documents))

wwater = water_relevant_documents['File Name'].tolist()
print(len(wwater))
wwater_1 = [w for w in wwater if w in combined_term_docs]
print(len(wwater_1))

0
282
0


In [88]:
common_livelihood_relevant_documents = [
    doc for _, doc in livelihood_relevant_documents.iterrows()
    if doc['File Name'] in combined_term_docs
]
print(len(common_livelihood_relevant_documents))

0


In [89]:
# Term Frequencies for the Term Coverage selected documents

client = MongoClient("mongodb://localhost:27017/")
db = client["transcripts"]
collection = db["complete_documents"]

document_names = pd.read_excel(r'TermCoverageListCombined.xlsx')
target_words = ["abatement", "neutrality", "carbon", "emissions", "nitro","nitrogen-efficient","algal","derogation","effluent","dualgas","amalgamate","nitrate","discharge","nitrite","run-off","nitrogen-based","amalgamations","dealgan","rivers","algae","eutrophication","amalgama","dualgais","bloom","waterways","amalgamated","nitrous","nitrogen","amalgamation","nitrogen-fixing","water","nitrates","inheriting","community","society","inheritances","herity","inherited","inherits","heritability","heritage","succession","inherit","thatcherite","anti-heritage","inheritance"]

query = {"File Name": {"$in": document_names["File Name"].tolist()}}
results = collection.find(query, {"File Name": 1, "Term Document Matrix": 1})

word_frequencies = []

for doc in results:
    file_name = doc["File Name"]
    term_matrix = doc.get("Term Document Matrix", {})
    
    frequencies = {word: term_matrix.get(word, 0) for word in target_words}
    frequencies["File Name"] = file_name
    word_frequencies.append(frequencies)

frequencies_df = pd.DataFrame(word_frequencies)

merged_df = document_names.merge(frequencies_df, on="File Name", how="left")

print(merged_df)

KeyError: 'File Name'

In [ ]:
merged_df.to_excel(r'merged_df.xlsx')

In [ ]:
print(f'Record Counts with Dairy: Carbon {len(dairy_carbon_doc)}, Water {len(dairy_water_doc)}, Livelihood {len(dairy_livelihood_doc)}')

In [ ]:
combined_list = dairy_carbon_doc + dairy_water_doc + dairy_livelihood_doc
print(len(set(combined_list)))

In [ ]:
relevant_doc = pd.DataFrame(set(combined_list))

In [ ]:
relevant_doc.to_csv(r'relevant_doc.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
output_file = "output.xlsx"

with pd.ExcelWriter(output_file) as writer:
    dairy_carbon_doc_with_frequencies.to_excel(writer, sheet_name="Sheet1", index=False)
    dairy_water_doc_with_frequencies.to_excel(writer, sheet_name="Sheet2", index=False)
    dairy_livelihood_doc_with_frequencies.to_excel(writer, sheet_name="Sheet3", index=False)

print(f"DataFrames exported to {output_file}")

In [ ]:
carbon_files = set(dairy_carbon_doc_with_frequencies['File Name'])
water_files = set(dairy_water_doc_with_frequencies['File Name'])
livelihood_files = set(dairy_livelihood_doc_with_frequencies['File Name'])

common_files = carbon_files & water_files & livelihood_files

common_carbon_records = dairy_carbon_doc_with_frequencies[dairy_carbon_doc_with_frequencies['File Name'].isin(common_files)]
common_water_records = dairy_water_doc_with_frequencies[dairy_water_doc_with_frequencies['File Name'].isin(common_files)]
common_livelihood_records = dairy_livelihood_doc_with_frequencies[dairy_livelihood_doc_with_frequencies['File Name'].isin(common_files)]

common_records = pd.merge(common_carbon_records, common_water_records, on='File Name', how='inner')
common_records = pd.merge(common_records, common_livelihood_records, on='File Name', how='inner')

# Export the result to a new sheet in the Excel file
with pd.ExcelWriter("common_files_output.xlsx") as writer:
    common_records.to_excel(writer, sheet_name="CommonFiles", index=False)

print("Records with common 'File Name' exported to 'common_files_output.xlsx'")

In [ ]:
docList = pd.read_csv(r'C:\Jupyter Notebook\Relevant Docs\relevant_doc.csv')
docList = docList['Doc'].tolist()
topHundred = [s.lower().replace('.pdf','') for s in docList[:100]]
print(len(topHundred))
print(topHundred[10])

In [ ]:
ca = [
    label if label.replace('.pdf','').lower() in topHundred else "" for label in dairy_carbon_doc_with_frequencies["File Name"]
]
print(ca)

In [ ]:
import matplotlib.pyplot as plt

carbon_x_labels = dairy_carbon_doc_with_frequencies["File Name"]
#carbon_x_labels = [label if label.replace('.pdf','').lower() in topHundred else "" for label in df["File Name"]]
carbon_categories = ["abatement", "neutrality", "carbon", "emissions"]

fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.2
x = range(len(carbon_x_labels))
for i, category in enumerate(carbon_categories):
    ax.bar(
        [p + i * bar_width for p in x], 
        dairy_carbon_doc_with_frequencies[category], 
        width=bar_width, 
        label=category
    )

ax.set_xticks([p + (len(carbon_categories) - 1) * bar_width / 2 for p in x])
ax.set_xticklabels(carbon_x_labels, rotation=45, ha="right")
ax.set_ylabel("Frequency Count")
ax.set_title("Frequency Count by File Name")
ax.legend(title="Categories")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

water_x_labels = dairy_water_doc_with_frequencies["File Name"]
water_categories = ["nitro","nitrogen-efficient","algal","derogation","effluent","dualgas","amalgamate","nitrate","discharge","nitrite","run-off","nitrogen-based","amalgamations","dealgan","rivers","algae","eutrophication","amalgama","dualgais","bloom","waterways","amalgamated","nitrous","nitrogen","amalgamation","nitrogen-fixing","water","nitrates"]

fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.2
x = range(len(water_x_labels))
for i, category in enumerate(water_categories):
    ax.bar(
        [p + i * bar_width for p in x], 
        dairy_water_doc_with_frequencies[category], 
        width=bar_width, 
        label=category
    )

ax.set_xticks([p + (len(water_categories) - 1) * bar_width / 2 for p in x])
ax.set_xticklabels(water_x_labels, rotation=45, ha="right")
ax.set_ylabel("Frequency Count")
ax.set_title("Frequency Count by File Name")
ax.legend(title="Categories")

plt.tight_layout()
plt.show()

In [ ]:
la = [
    label if label[:-4] in topHundred else "" for label in dairy_livelihood_doc_with_frequencies["File Name"]
]
print(la)

In [ ]:
import matplotlib.pyplot as plt

livelihood_x_labels = dairy_livelihood_doc_with_frequencies["File Name"]
livelihood_categories = ["inheriting","community","society","inheritances","herity","inherited","inherits","heritability","heritage","succession","inherit","thatcherite","anti-heritage","inheritance"]
fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.2
x = range(len(livelihood_x_labels))
for i, category in enumerate(livelihood_categories):
    ax.bar(
        [p + i * bar_width for p in x], 
        dairy_livelihood_doc_with_frequencies[category], 
        width=bar_width, 
        label=category
    )

ax.set_xticks([p + (len(livelihood_categories) - 1) * bar_width / 2 for p in x])
ax.set_xticklabels(livelihood_x_labels, rotation=45, ha="right")
ax.set_ylabel("Frequency Count")
ax.set_title("Frequency Count by File Name")
ax.legend(title="Categories")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.plot(dairy_carbon_doc_with_frequencies['Total Freq'], label='Carbon')
plt.plot(dairy_water_doc_with_frequencies['Total Freq'], label='Water')
plt.plot(dairy_livelihood_doc_with_frequencies['Total Freq'], label='Livelihood')

plt.xlabel("Index")
plt.ylabel("Total Frequency")
plt.title("Comparison of Total Frequencies")

plt.legend()

plt.grid(True, linestyle="--", alpha=0.5)

plt.show()

In [ ]:
output_file = "threshold_total_Frequency.xlsx"

with pd.ExcelWriter(output_file) as writer:
    dairy_carbon_doc_with_frequencies.to_excel(writer, sheet_name="Carbon", index=False)
    dairy_water_doc_with_frequencies.to_excel(writer, sheet_name="Water", index=False)
    dairy_livelihood_doc_with_frequencies.to_excel(writer, sheet_name="Livelihood", index=False)

print(f"DataFrames exported to {output_file}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Ensure sorting by Year before plotting
dairy_carbon_doc_with_frequencies = dairy_carbon_doc_with_frequencies.sort_values(by="Year")
dairy_water_doc_with_frequencies = dairy_water_doc_with_frequencies.sort_values(by="Year")
dairy_livelihood_doc_with_frequencies = dairy_livelihood_doc_with_frequencies.sort_values(by="Year")

# Scatter plot for each dataset with Year on x-axis
plt.scatter(dairy_carbon_doc_with_frequencies["Year"], dairy_carbon_doc_with_frequencies["Total Freq"], label="Carbon", alpha=0.4)
plt.scatter(dairy_water_doc_with_frequencies["Year"], dairy_water_doc_with_frequencies["Total Freq"], label="Water", alpha=0.4)
plt.scatter(dairy_livelihood_doc_with_frequencies["Year"], dairy_livelihood_doc_with_frequencies["Total Freq"], label="Livelihood", alpha=0.4)

# Labels and Title
plt.xlabel("Year")
plt.ylabel("Total Frequency")
plt.title("Comparison of Total Frequencies Over the Years")

# Display legend
plt.legend()

# Grid for better readability
plt.grid(True, linestyle="--", alpha=0.5)

# Show the plot
plt.show()
